In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [44]:
checkindf = pd.read_csv(r'C:\Users\NanaYawDarko\door_access_accra\data_exports\CHECKINOUT.csv')
userdf = pd.read_csv(r'C:\Users\NanaYawDarko\door_access_accra\data_exports\USERINFO.csv')
eventlogdf= pd.read_csv(r'C:\Users\NanaYawDarko\door_access_accra\data_exports\acc_monitor_log.csv')
departmentdf = pd.read_csv(r'C:\Users\NanaYawDarko\door_access_accra\data_exports\DEPARTMENTS.csv')
machinesdf = pd.read_csv(r'C:\Users\NanaYawDarko\door_access_accra\data_exports\Machines.csv')


In [ ]:
# get the columns of interest from the userdf
userdf_columns = ['USERID','name','lastname','email','DEFAULTDEPTID']
userdf = userdf[userdf_columns]
#Rename the columns to be more descriptive
userdf.rename(columns={'USERID':'userid','name':'firstname','email':'Email','DEFAULTDEPTID':'deptid'}, inplace=True)
userdf.describe()

,userid
count,313.000000
mean,168.367412
std,95.485311
min,1.000000
25%,87.000000
50%,171.000000
75%,251.000000
max,329.000000


In [ ]:
# get the columns of interest from the checkindf
# checkindf_columns = ['USERID','CHECKTIME','LOGID',]
# checkindf = checkindf[checkindf_columns]
# checkindf.rename(columns={'USERID':'userid','CHECKTIME':'checktime','LOGID':'logid'}, inplace=True)
checkindf['checktime'] = pd.to_datetime(checkindf['checktime'], format='%Y-%m-%d %H:%M:%S')

# filter the checkindf to only include records from 2024 to now
checkindf = checkindf[checkindf['checktime'] == 2024]
checkindf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   userid     0 non-null      int64         
 1   checktime  0 non-null      datetime64[ns]
 2   logid      0 non-null      int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 0.0 bytes


In [65]:
# get the columns of interest from eventlogdf
# eventlog_columns = ['id','time','device_name','state','event_type','event_point_name']
# eventlogdf = eventlogdf[eventlog_columns]
# eventlogdf.rename(columns={'id':'logid','time':'logtime'}, inplace=True)
eventlogdf['logtime'] = pd.to_datetime(eventlogdf['logtime'], format='%Y-%m-%d %H:%M:%S')
eventlogdf.dtypes

logid                        int64
logtime             datetime64[ns]
device_name                 object
state                        int64
event_type                   int64
event_point_name            object
dtype: object

In [86]:
# left join userdf to checkindf on userid
usercheckindf = userdf.merge(checkindf, on='userid', how='inner')

# usercheckindf.groupby('name').size().reset_index(name='count').sort_values(by='count', ascending=False).head(10)
usercheckindf.sort_values(by='checktime', ascending=True).head(100).reset_index(drop=True)

,userid,name,lastname,email,checktime,logid
